# Intelligent Imaging 2021 Summer Bootcamp: Day 1
- What you will need: 
  - A google Drive account
  - A copy of this folder in your G Drive [google drive](https://drive.google.com/drive/folders/1xxgXZt42XEexnGdKzHr69iTxE298ADIY?)
- Some experience with python (pair up!!)

## Loading medical imaging data

### Loading image from google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
knee_dir = '/content/gdrive/MyDrive/intelligent_imaging_bootcamp_2021/Day_1/images/dicom/knee'
heart_dir = '/content/gdrive/MyDrive/intelligent_imaging_bootcamp_2021/Day_1/images/nifti/Task02_Heart/imagesTr'
chest_file =  '/content/gdrive/MyDrive/intelligent_imaging_bootcamp_2021/Day_1/images/dicom/chest-xray/1-3.dcm'
chest_jpeg = '/content/gdrive/MyDrive/intelligent_imaging_bootcamp_2021/Day_1/images/jpeg/chest_Xray.jpg'

## Setup environment

In [ ]:
!python -c "import monai" || pip install -q "monai-weekly[itk, pillow]"
!pip install pydicom

In [ ]:
# from PIL import Image
# import tempfile
# from monai.data import ITKReader, PILReader
import itk
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil


# Load images with default image reader
MONAI automatically choose readers based on the supported suffixes and in below order:
- User specified reader at runtime when call this loader.
- Registered readers from the latest to the first in list.
- Default readers: (nii, nii.gz -> NibabelReader), (png, jpg, bmp -> PILReader),
  (npz, npy -> NumpyReader), (others -> ITKReader).


# Loading Medical Imaging Data

## Load dicom data

For more information, see this [blog post](https://mscipio.github.io/post/read_dicom_files_in_python/)

In [ ]:
import pydicom as dicom
ds = dicom.read_file(chest_file)
ds

In [ ]:
#to see all dicom keys available, use `dir`
ds.dir()[:10]

In [ ]:
#you can also search by partial key substrings
ds.dir('pa')

In [ ]:
#to get an element in the dicom file, use `data_element`
ds.data_element("BodyPartExamined")

### Load image data

In [ ]:
from monai.transforms import *

In [ ]:
!ls $knee_dir

In MONAI, images are loaded with `LoadImage()(path)` where `path` is the location of the image (if 1 image) or folder if multiple slices

In [ ]:
data, meta = LoadImage()(knee_dir)
print(data.shape)

In [ ]:
#visualize it (here, slice #12)
slice=12
plt.imshow(data[slice,:,:]);

## Load nifti data

What is nifti?

In [ ]:
nifti_files = os.listdir(heart_dir)
nifti_paths = [heart_dir + '/' + file for file in nifti_files]

In [ ]:
data, meta = LoadImage()(nifti_paths[0])

In [ ]:
data.shape

In [ ]:
#visualize data
slice=60
plt.matshow(data[:,:,slice]);

Challenge: can you rotate this image to look "normal"?

In [ ]:
#@title
slice=60
plt.matshow(data[:,::-1,slice].T);

## Load jpeg

In [ ]:
data, meta = LoadImage()(chest_jpeg)

In [ ]:
data.shape

In [ ]:
plt.imshow(data[:,:,0])

Q: What are the 3 "slices" in the jpeg image?

# Manipulating Data

MONAI uses `Transforms` to modify data. See the [Transforms Documentation Page](https://docs.monai.io/en/latest/transforms.html)

If several transforms are needed, you can combine them using `Compose`.

Since in general loading an image returns the image and some *metadata* associated with it, it makes sense to read the image as a dictionary so we can apply transforms only to the *image* (and not to the text).

Every transform in MONAI also has a dictionary-based version to make it easier to apply a transform to a particular key only. This is also useful if you want to apply a transform to the training set but not to the test set, for instance.

In [ ]:
#to load the same chest image above as a dictionary:
LoadImaged(keys=['image'])({'image': chest_jpeg})

### Center cropping an image with `Resized`

In [ ]:
transform = Compose([
    LoadImaged(keys="image"),
    Resized(keys="image", spatial_size=[64, 64]), # this is the transform !!!
    ToTensord("image"),
])
test_data = {"image": knee_dir}
result = transform(test_data)
plt.imshow(result['image'][12,:,:]);

### Randomly crop an image with `RandSpatialCropd`
Run the cell below several times and see how the solution changes.

In [ ]:
transform = Compose([
    LoadImaged(keys="image"),
    RandSpatialCropd(keys="image", roi_size=[64,64], max_roi_size=[64, 64]),
    ToTensord("image"),
])
test_data = {"image": knee_dir}
result = transform(test_data)
plt.imshow(result['image'][12,:,:]);

## Pad image

In [ ]:
transform = Compose([
    LoadImaged(keys="image"),
    SpatialPadd(keys="image", spatial_size=[700, 700]),
    ToTensord("image"),
])
test_data = {"image": knee_dir}
result = transform(test_data)
plt.imshow(result['image'][12,:,:]);

## Normalizing data
- Mean = 0.0
- Standard deviation = 1.0

In [ ]:
transform = Compose([
    LoadImaged(keys="image"),
    Resized(keys="image", spatial_size=[64, 64]),
    NormalizeIntensityd(keys="image"),
    ToTensord("image"),
])
test_data = {"image": knee_dir}
result = transform(test_data)
plt.imshow(result['image'][12,:,:]);

Q: Can you use `ThresholdIntensityd` to threshold the normalized image?

In [ ]:
#@title
transform = Compose([
    LoadImaged(keys="image"),
    Resized(keys="image", spatial_size=[64, 64]),
    NormalizeIntensityd(keys="image"),
    ThresholdIntensityd(keys="image", threshold=1.0),
    ToTensord("image"),
])
test_data = {"image": knee_dir}
result = transform(test_data)
plt.imshow(result['image'][12,:,:]);

# Break out session
- Load Brain data
- For Training data:
   - Normalize
   - Augment: crop, flip, rotate
-  